LangChain 프롬프트 템플릿
```
LLM 프롬프트를 동적으로 구성하고 재 사용할 수 있도록 해 주는 도구
    단일입력 : 하나의 변수로 구성된 프롬프트 템플릿
    다중입력 : 둘 이상의 변수를 사용하는 템플릿
    ChatPromptTemplate 역할 기반 프롬프트 : 시스템/사용자 역활별 프롬프트 구성 .from_message()
    PartialPromptTemplate 활용 : 프롬프트 일부를 미리 고정하고 부분포멧팅을 사용(ex 시스템 메시지는 고정,,)
    프롬프트 출력 및 체인 실행 : LCEL
    프롬프트 작성 팁: 주의사항 및 모범사례
```

In [3]:
#  환경설정
%pip install --quiet langchain langchain-openai python-dotenv 

Note: you may need to restart the kernel to use updated packages.


In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
# 단일 프롬프트 사용
from langchain.prompts import PromptTemplate
# 템플릿 문자열 정의
template_str = (
    "당신은 최고 수준의 마케팅 카피라이터 입니다.\n"
    "아래 제품의 매력적인 홍보문구를 100자 내외로 작성해 주세요.\n\n"
    "제품명 : {product_name}"
)
# PromptTemplate 객체 생성
product_prompt = PromptTemplate.from_template(template_str)
# 탬플릿에 값 채우기
formatted_prompt = product_prompt.format(product_name="이클립스")
print(formatted_prompt)

당신은 최고 수준의 마케팅 카피라이터 입니다.
아래 제품의 매력적인 홍보문구를 100자 내외로 작성해 주세요.

제품명 : 이클립스


In [12]:
# 프롬프트 | llm  -> invoke
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# Runnable 객체 생성  LCEL
chain = product_prompt | llm
result = chain.invoke({"product_name": "커피"})
print(result.content)

"하루의 시작을 특별하게! 깊고 풍부한 향의 커피 한 잔으로 당신의 순간을 깨워보세요. 진정한 여유를 느껴보세요!"


In [14]:
# 다중 입력
# 다중입력 템플릿 문자열 정의
multi_template_str = (
    "아래는 뉴스 기사 제목과 키워드 입니다.\n"
    "이 정보를 바탕으로 한 문단으로 구성된 간략한 요약문을 작성하세요\n\n"
    "제목:{title}\n"
    "키워드:{keyword}"
)
# 프롬프트 탬플릿 작성
summary_prompt = PromptTemplate(template=multi_template_str, input_variables=["title","keyword"])
# 포멧팅을 통해 프롬프트 값 확인
sample_title = '인공지능 기술의 발전과 미래'
sample_keyword = '머신러닝,딥러닝,LLM,랭체인,산업 혁신'
formatted_summary_promptt = summary_prompt.format(title = sample_title, keyword=sample_keyword)
print(formatted_summary_promptt)

아래는 뉴스 기사 제목과 키워드 입니다.
이 정보를 바탕으로 한 문단으로 구성된 간략한 요약문을 작성하세요

제목:인공지능 기술의 발전과 미래
키워드:머신러닝,딥러닝,LLM,랭체인,산업 혁신


In [16]:
# LCEL 출력
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
result_summary = (summary_prompt | llm | parser).invoke({
    'title':  sample_title,
    'keyword':sample_keyword
})
result_summary

'인공지능 기술의 발전은 머신러닝과 딥러닝의 혁신을 통해 가속화되고 있으며, 특히 대규모 언어 모델(LLM)과 랭체인 기술이 주목받고 있습니다. 이러한 기술들은 다양한 산업 분야에서 혁신을 이끌어내고 있으며, 기업들은 이를 활용하여 효율성을 높이고 새로운 비즈니스 모델을 창출하고 있습니다. 앞으로 인공지능은 더욱 발전하여 우리의 삶과 산업 전반에 걸쳐 큰 변화를 가져올 것으로 기대됩니다.'

In [17]:
# ChatPromptTemplate과 역활 기반 프롬프트
# 시스템/사용자/어시스턴트 역활(role)
# 시스템 메시지: 모델의 동작을 지시
# 사용자 메시지: 실제 사용자의 입력
# 어시스턴트 메시지: 이전 모델이 응답한 내용이 있드면 대화맥락유지를 위해 활용용

In [20]:
from langchain.prompts import ChatPromptTemplate
system_message = ("당신은 python 분야의 뛰어난 전문가이자 조언자 입니다."
                  "사용자의 프로그래밍 질문에 대해 친절하고 이해하기 쉽게 답변해 주세요")
user_message = "{question}"
chat_prompt = ChatPromptTemplate.from_messages([
    ("system",system_message),
    ("user",user_message)
])
# 템플릿을 이용해서 문장을 완성해 보세요 
sample_question = '파이썬의 클래스에 대해서 설명해주세요'
formated_message = chat_prompt.format_messages(question=sample_question)
print(formated_message)
# 파이프라인을 이용해서 llm호출 및 출력
answer = (chat_prompt | llm | parser).invoke({'question' : sample_question})
print(answer)

[SystemMessage(content='당신은 python 분야의 뛰어난 전문가이자 조언자 입니다.사용자의 프로그래밍 질문에 대해 친절하고 이해하기 쉽게 답변해 주세요', additional_kwargs={}, response_metadata={}), HumanMessage(content='파이썬의 클래스에 대해서 설명해주세요', additional_kwargs={}, response_metadata={})]
파이썬의 클래스는 객체 지향 프로그래밍(OOP)의 기본 개념 중 하나로, 객체를 생성하기 위한 청print(blueprint)입니다. 클래스는 속성(데이터)과 메서드(함수)를 포함할 수 있으며, 이를 통해 관련된 데이터와 기능을 하나의 단위로 묶을 수 있습니다.

### 클래스의 기본 구조

클래스를 정의하려면 `class` 키워드를 사용합니다. 다음은 간단한 클래스의 예입니다:

```python
class Dog:
    # 생성자 메서드
    def __init__(self, name, age):
        self.name = name  # 인스턴스 변수
        self.age = age    # 인스턴스 변수

    # 메서드
    def bark(self):
        return f"{self.name} says woof!"

# 클래스의 인스턴스 생성
my_dog = Dog("Buddy", 3)

# 메서드 호출
print(my_dog.bark())  # 출력: Buddy says woof!
```

### 주요 개념

1. **생성자 (`__init__` 메서드)**:
   - 클래스의 인스턴스가 생성될 때 호출되는 특별한 메서드입니다.
   - 인스턴스 변수를 초기화하는 데 사용됩니다.

2. **인스턴스 변수**:
   - `self` 키워드를 통해 클래스의 인스턴스에 속하는 변수를 정의합니다.
   - 각 인스턴스마다 고유한 값을 가질 수 있습니다.

3. **메서드**:
   - 클래스 내부에 정의된 함수로,